In [1]:
import subprocess, shutil, datetime
from pathlib import Path

import numpy as np

from vtu import PyVtu
from ts_auto_wrapper import TSWrapper

In [2]:
ts=TSWrapper(Path("/opt/workspace/msc_project/cluster-trisurf"))

In [3]:
test_folder=None
tape=None
extra_args = []
timeout = 120 # seconds to timeout for each `trisurf` test

### Make a new folder if there isn't one ###

In [4]:
new_folder=None
if not test_folder:
    # make up some appropriate folder
    now = datetime.datetime.now()
    ret = subprocess.run(['git', 'status'],cwd=ts.path_to_trisurf,capture_output=True)
    branch = ret.stdout.decode().partition('On branch ')[-1].splitlines()[0].strip()
    new_folder = f'{now.year:04}_{now.month:02}_{now.day:02}_{branch}'
    test_folder = Path('.')/"trisurf_test"/new_folder
new_folder

'2023_05_24_main'

In [5]:
if not test_folder.exists():
    test_folder.mkdir()
    test_folder.joinpath('test_results').mkdir()
elif new_folder is not None:
    raise ValueError(f'folder {test_folder} for today already exists: Rename or delete to run test')

### copy the tape to the new folder ###

In [6]:
if tape is None:
    tape=ts.path_to_trisurf/"src/tape"
shutil.copy(tape,test_folder/"tape")

PosixPath('trisurf_test/2023_05_24_main/tape')

## Check force from tape ##

In [7]:
ret = subprocess.run([ts.path_to_trisurf/"src/trisurf", "--force-from-tape", *extra_args],
                     cwd=test_folder, timeout=timeout, capture_output=True)

In [8]:
with open(test_folder/"test_results"/"force_stdout.txt",'w') as f: 
    f.write(ret.stdout.decode())
print(ret.stdout.decode())

[2023-05-24 18:37:21] TRISURF-NG v. a124e32, compiled on: May 24 2023 16:05:49.
[2023-05-24 18:37:21] Programming done by: Samo Penic and Miha Fosnaric
[2023-05-24 18:37:21] Released under terms of GPLv3
[2023-05-24 18:37:21] Starting program...

[2023-05-24 18:37:21] ************************************************
[2023-05-24 18:37:21] **** Generating initial geometry from tape *****
[2023-05-24 18:37:21] ************************************************

[2023-05-24 18:37:21] Starting initial_distribution on vesicle with 10 shells!...
[2023-05-24 18:37:21] initial_distribution finished!
[2023-05-24 18:37:21] using debug curvature model 15 (use new energy): set to 7 to use the old energy method
[2023-05-24 18:37:21] simulation seed 1684942641
[2023-05-24 18:37:21] Setting volume V0=1041.99378875998718286
[2023-05-24 18:37:21] Setting area A0=623.53829072480641571
[2023-05-24 18:37:24] Done 1 out of 10 iterations (x 1000 MC sweeps).
[2023-05-24 18:37:26] Done 2 out of 10 iterations (x 

## Check start from binary dump ##

In [9]:
ret = subprocess.run([ts.path_to_trisurf/"src/trisurf","--reset-iteration-count", *extra_args],
                     cwd=test_folder, timeout=timeout, capture_output=True)

In [10]:
with open(test_folder/"test_results"/"reset_stdout.txt",'w') as f: 
    f.write(ret.stdout.decode())
print(ret.stdout.decode())

[2023-05-24 18:37:44] TRISURF-NG v. a124e32, compiled on: May 24 2023 16:05:49.
[2023-05-24 18:37:44] Programming done by: Samo Penic and Miha Fosnaric
[2023-05-24 18:37:44] Released under terms of GPLv3
[2023-05-24 18:37:44] Starting program...

[2023-05-24 18:37:44] **********************************************************************
[2023-05-24 18:37:44] **** Recreating vesicle from dump file and continuing simulation *****
[2023-05-24 18:37:44] **********************************************************************

[2023-05-24 18:37:45] using debug curvature model 15 (use new energy): set to 7 to use the old energy method
[2023-05-24 18:37:45] simulation seed 1684942665
[2023-05-24 18:37:45] Setting volume V0=1900.94212002665403816
[2023-05-24 18:37:45] Setting area A0=769.81037142648301597
[2023-05-24 18:37:47] Done 1 out of 10 iterations (x 1000 MC sweeps).
[2023-05-24 18:37:49] Done 2 out of 10 iterations (x 1000 MC sweeps).
[2023-05-24 18:37:51] Done 3 out of 10 iterations (x

## Checking start from vtu ##

In [19]:
test_folder.joinpath('.status').unlink() # remove status so it resets iterations

In [20]:
ret = subprocess.run([ts.path_to_trisurf/"src/trisurf",
                      "--restore-from-vtk",'timestep_000000.vtu', *extra_args],
                     cwd=test_folder, timeout=timeout, capture_output=True)

In [21]:
with open(test_folder/"test_results"/"restore_stdout.txt",'w') as f: 
    f.write(ret.stdout.decode())
print(ret.stdout.decode())

[2023-05-24 18:44:43] TRISURF-NG v. a124e32, compiled on: May 24 2023 16:05:49.
[2023-05-24 18:44:43] Programming done by: Samo Penic and Miha Fosnaric
[2023-05-24 18:44:43] Released under terms of GPLv3
[2023-05-24 18:44:43] Starting program...

[2023-05-24 18:44:43] ************************************************
[2023-05-24 18:44:43] **** Restoring vesicle from VTK points list ****
[2023-05-24 18:44:43] ************************************************

[2023-05-24 18:44:43] No .status file. The iteration count will start from 0
[2023-05-24 18:44:43] using debug curvature model 15 (use new energy): set to 7 to use the old energy method
[2023-05-24 18:44:43] simulation seed 1684943083
[2023-05-24 18:44:43] Setting volume V0=1873.60715166322597725
[2023-05-24 18:44:43] Setting area A0=760.56143968803814914
[2023-05-24 18:44:46] Done 1 out of 10 iterations (x 1000 MC sweeps).
[2023-05-24 18:44:48] Done 2 out of 10 iterations (x 1000 MC sweeps).
[2023-05-24 18:44:50] Done 3 out of 10 it

## More tests ##

In [13]:
raise RuntimeError('up to here')

RuntimeError: up to here

In [85]:
fileloc = (test_folder/"timestep_000000.vtu")

In [86]:
vesicle = ts.parseDump(fileloc)

In [87]:
v = PyVtu(fileloc)
v2 = PyVtu(fileloc)

In [88]:
def iter_xlist(vesiclePtr,xlist_name='vlist'):
    lst = vesiclePtr.contents.__getattribute__(xlist_name)
    if xlist_name=='clist':
        n, lst = lst.contents.cellno, lst.contents.cell
    else:
        n, lst = lst.contents.n, lst.contents.__getattribute__(lst.contents._fields_[0][0])
    for i in range(n):
        yield lst[i].contents


In [102]:
vtx0=vesicle.contents.vlist.contents.vtx[0].contents

In [111]:
def fake_pyvtu(v,vesiclePtr):
    v.pos[:] = [(x.x,x.y,x.z) for x in iter_xlist(vesicle)]
    v.normal[:] = [(x.nx,x.ny,x.nz) for x in iter_xlist(vesicle)]
    v.force[:] =[(x.f*x.fx,x.f*x.fy,x.f*x.fz) for x in iter_xlist(vesicle)]
    v.director[:] = [(x.dx,x.dy,x.dz) for x in iter_xlist(vesicle)]
    v.eig0[:] = [x.eig0[:] for x in iter_xlist(vesicle)]
    v.eig1[:] = [x.eig1[:] for x in iter_xlist(vesicle)]
    v.eig2[:] = [x.eig2[:] for x in iter_xlist(vesicle)]
    v.e[:] = [x.energy for x in iter_xlist(vesicle)]
    v.c0[:] = [x.c for x in iter_xlist(vesicle)]
    v.w[:] = [x.w for x in iter_xlist(vesicle)]
    v.f0[:] = [x.f for x in iter_xlist(vesicle)]
    v.ad_w[:] = [x.ad_w for x in iter_xlist(vesicle)]
    v.type[:] = [ts.byte_to_int(x.type) for x in iter_xlist(vesicle)]
    v.eigenvalue_0[:] = [x.eig_v0 for x in iter_xlist(vesicle)]
    v.eigenvalue_1[:] = [x.eig_v1 for x in iter_xlist(vesicle)]
    v.eigenvalue_2[:] = [x.eig_v2 for x in iter_xlist(vesicle)]
    v.k[:] = [x.xk for x in iter_xlist(vesicle)]
    v.k2[:] = [x.xk2 for x in iter_xlist(vesicle)]
    v.mean_curvature[:] = [x.mean_curvature for x in iter_xlist(vesicle)]
    v.gaussian_curvature[:] = [x.gaussian_curvature for x in iter_xlist(vesicle)]
    v.mean_curvature2[:] = [x.mean_curvature2 for x in iter_xlist(vesicle)]
    v.gaussian_curvature2[:] = [x.gaussian_curvature2 for x in iter_xlist(vesicle)]
    v._arrays['neigh_no'] = np.array([x.neigh_no for x in iter_xlist(vesicle)])
    

In [113]:
fake_pyvtu(v2,vesicle)

In [116]:
v.pos

array([[ 0.21752589,  0.21579287,  6.41768125],
       [ 0.74362265,  1.89236547,  5.89730745],
       [ 1.13641831,  0.7806501 ,  6.07100536],
       ...,
       [ 3.56339245,  1.46699506, -6.37772209],
       [-0.53287086,  2.13181142, -6.77468786],
       [ 0.07765391,  0.66106834, -7.08689121]])

In [117]:
v2.pos

array([[ 0.21752589,  0.21579287,  6.41768125],
       [ 0.74362265,  1.89236547,  5.89730745],
       [ 1.13641831,  0.7806501 ,  6.07100536],
       ...,
       [ 3.56339245,  1.46699506, -6.37772209],
       [-0.53287086,  2.13181142, -6.77468786],
       [ 0.07765391,  0.66106834, -7.08689121]])

In [118]:
[(k,np.allclose(v._arrays[k],v2._arrays[k])) for k in v._arrays]

[('pos', True),
 ('blist', True),
 ('tlist', True),
 ('indices', True),
 ('type', True),
 ('c0', True),
 ('w', True),
 ('f0', True),
 ('ad_w', True),
 ('d0', True),
 ('mean_curvature', True),
 ('gaussian_curvature', True),
 ('mean_curvature2', True),
 ('gaussian_curvature2', True),
 ('eigenvalue_0', True),
 ('eigenvalue_1', True),
 ('eigenvalue_2', True),
 ('k', True),
 ('k2', True),
 ('e', False),
 ('mean_energy', True),
 ('gaussian_energy', True),
 ('mean_energy2', True),
 ('gaussian_energy2', True),
 ('normal', True),
 ('normal_unused_debug', True),
 ('force', False),
 ('director', False),
 ('eig0', True),
 ('eig1', True),
 ('eig2', True),
 ('bonding_energy', True),
 ('face_normal', True)]

In [131]:
strange_f=v.indices[~(np.isclose(v.force,v2.force).any(axis=1))]
strange_d=v.indices[~(np.isclose(v.director,v2.director).any(axis=1))]

In [132]:
(v2.force[strange_f]*v.force[strange_f]).sum(axis=1)

array([0.99787833, 0.99982396, 0.99995001, 0.99979397, 0.99993833,
       0.99999975, 0.99989288, 0.9999804 , 0.99990011, 0.9996873 ,
       0.99964804, 0.99999988, 0.99976618, 0.99986417, 0.99985958,
       0.99999549, 0.99982243, 0.99999984, 0.9995469 , 0.99999917,
       0.99970687, 0.99995449, 0.99827257, 0.99956051, 0.99579193,
       0.99995669, 0.99936812, 0.99911567, 0.99931747, 0.99999998,
       0.99997637, 0.999928  , 0.9989933 , 0.99991203, 0.99801203,
       0.9999424 , 0.9999885 , 0.99951571, 0.99919313, 0.99903415,
       0.99980649, 0.99759985, 0.99999452, 0.99982352, 0.99889831])

In [133]:
(v2.director[strange_d]*v.director[strange_d]).sum(axis=1)

array([0.99999981, 1.        , 1.        , 0.99999999, 0.99999999,
       0.99999999, 0.99999999, 1.        , 0.99999998, 0.99999993,
       0.99999999, 1.        , 1.        , 0.99999999, 0.99999997,
       0.99999999, 1.        , 0.99999999, 0.99999997, 0.99999996,
       1.        , 1.        , 1.        , 0.99999992, 1.        ,
       0.99999999, 0.99999994, 1.        , 1.        , 0.99999999,
       1.        , 0.99999993, 0.99999994, 0.99999999, 0.99999999,
       1.        , 0.99999999, 0.9999999 , 1.        , 0.99999997,
       1.        , 0.99999993, 1.        , 1.        , 1.        ,
       1.        , 0.99999975, 0.99999999, 0.99999999, 0.99999997,
       1.        , 1.        , 0.99999999, 1.        , 1.        ,
       0.99999999, 1.        , 1.        , 1.        , 1.        ,
       0.99999999, 0.99999992, 1.        , 0.99999999, 1.        ,
       0.99999998, 0.99999875, 1.        , 0.99999998, 1.        ,
       0.99999999, 0.99999999, 0.99999998, 0.99999983, 0.99999

In [138]:
(v.normal*v2.normal)[:].sum(axis=1)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1.

In [139]:
vtx = vesicle.contents.vlist.contents.vtx[0]

In [140]:
vtxc = vtx.contents

In [141]:
vtxm, vtxp = [x.contents for x in vtxc.neigh[:2]]

In [142]:
vtxm.idx, vtxp.idx

(5, 2)

In [143]:
t = vtxc.tristar[0].contents

In [144]:
v2.pos[[0,4,8]]

array([[ 0.21752589,  0.21579287,  6.41768125],
       [-0.80000475, -0.41208571,  6.40364975],
       [ 2.47828326,  0.37261328,  5.84472118]])

In [145]:
lm = np.array([vtxm.x-vtxc.x,vtxm.y-vtxc.y,vtxm.z-vtxc.z])
lp = np.array([vtxp.x-vtxc.x,vtxp.y-vtxc.y,vtxp.z-vtxc.z])

In [146]:
lm,lp

(array([-0.71721103,  0.71304839, -0.22563655]),
 array([ 0.91889242,  0.56485723, -0.34667589]))

In [147]:
v.pos[8]-v.pos[0]

array([ 2.26075737,  0.15682041, -0.57296007])

In [148]:
tnorm = [[x.xnorm,x.ynorm,x.znorm] for i in range(vtxc.tristar_no) if (x:=vtxc.tristar[i].contents)]

In [149]:
np.array(tnorm).mean(axis=0)

array([-0.13701105, -0.05955435, -0.95267044])

In [150]:
vtxc.nx,vtxc.ny,vtxc.nz

(0.1309156823222204, 0.0965561404850986, 0.9866802905991024)

In [151]:
N = np.array((t.xnorm,t.ynorm,t.znorm))

In [152]:
np.cross(lm,lp)@N

1.160416320935553

In [153]:
vtx_f=vesicle.contents.vlist.contents.vtx[5].contents

In [154]:
vtx_f.w

0.0

In [155]:
vertices = vesicle.contents.vlist.contents.vtx

In [156]:
vertices = [vesicle.contents.vlist.contents.vtx[i].contents 
            for i in range(vesicle.contents.vlist.contents.n)]

In [157]:
v.normal

array([[ 0.13091568,  0.09655614,  0.98668029],
       [ 0.04995019,  0.07358923,  0.99603695],
       [ 0.14846778,  0.08578043,  0.98518985],
       ...,
       [ 0.47328716,  0.14934419, -0.86815642],
       [-0.20588545,  0.25078991, -0.94589408],
       [-0.0139906 ,  0.09004964, -0.99583901]])

In [158]:
normals = np.array([[vtx.nx,vtx.ny,vtx.nz] for vtx in vertices])

In [159]:
areas = np.array([ts.area_vtx(ts.pointer(vtx)) for vtx in vertices])

In [160]:
areas

In [161]:
ts.laplace_beltrami_curvature_energy(vesicle,ts.pointer(vertices[2]))

True